In [1]:
import numpy as np
from copy import deepcopy
import time
from pycromanager import Acquisition, Bridge, multi_d_acquisition_events

In [2]:
bridge = Bridge()
mmc = bridge.get_core()
mmStudio = bridge.get_studio()

C:\Users\labelfree\.conda\envs\acq\lib\site-packages\pycromanager\core.py:258: UserWarning: Version mistmatch between Java ZMQ server and Python client. 
Java ZMQ server version: 4.0.0
Python client expected version: 4.1.0
 To fix, update to BOTH latest pycromanager and latest micro-manager nightly build
  warnings.warn(


In [3]:
mm_pos_list = mmStudio.get_position_list_manager().get_position_list()
number_of_positions = mm_pos_list.get_number_of_positions()

xyz_position_list = [[mm_pos_list.get_position(i).get_x(),
                      mm_pos_list.get_position(i).get_y(),
                      mm_pos_list.get_position(i).get_z()] \
                    for i in range(number_of_positions)]

xyz_position_list = np.around(xyz_position_list, decimals=3)

epi_positions = xyz_position_list[:3]
lf_epi_positions = xyz_position_list[3:]

# epi_positions = [xyz_position_list[0]]
# lf_epi_positions = [xyz_position_list[1]]

xyz_position_list

array([[ -9721.45 ,  12610.97 ,   4458.18 ],
       [ -9474.   ,  12607.08 ,   4458.2  ],
       [ -9382.24 ,  12973.58 ,   4458.3  ],
       [-10099.11 ,  13008.07 ,   4457.6  ],
       [-10748.13 ,  13060.17 ,   4457.14 ],
       [-11173.99 ,  13168.879,   4456.86 ]])

In [4]:
epi_channel = {'group': 'Master Channel', 'config': 'Epi'}
epi_exposure = 50

lf_channels = [{'group': 'Master Channel', 'config': f'LF-State{i}'} for i in range(1)]
lf_exposure = 5

# relative z
z_start = -2.5
z_end = 2.5
z_step = 0.25
z_range = np.arange(z_start, z_end + z_step, z_step)

num_time_points  = 72
time_interval_s  = 300  # in seconds

In [5]:
events = []
num_epi_positions = np.shape(epi_positions)[0]

for t in range(num_time_points):
    
    # epi-only positions, z stack events are created by autofocus function
    for p_idx, p in enumerate(epi_positions):
        events.append(
            {
                'axes': {'time': t, 'position':p_idx, 'z':0},
                'min_start_time': t*time_interval_s,
                'x': p[0],
                'y': p[1],
                'z': p[2],
                'channel': epi_channel,
                'exposure': epi_exposure,
                'keep_shutter_open': False,
            })

    # epi and lf positions, z stack events are created by autofocus function
    for p_idx, p in enumerate(lf_epi_positions):
        
        # epi channel
        events.append(
            {
                'axes': {'time': t,'position':p_idx+num_epi_positions, 'z':0},
                'min_start_time': t*time_interval_s,
                'x': p[0],
                'y': p[1],
                'z': p[2],
                'channel': epi_channel,
                'exposure': epi_exposure,
                'keep_shutter_open': False
            })
        
        # lf channels, CZ order
        for ch in lf_channels:
            events.append(
                {
                    'axes': {'time': t,'position':p_idx+num_epi_positions, 'z':0},
                    'min_start_time': t*time_interval_s,
                    'x': p[0],
                    'y': p[1],
                    'z': p[2],
                    'channel': ch,
                    'exposure': lf_exposure,
                    'keep_shutter_open': False
                })

In [6]:
num_events = len(events)

def autofocus_fn(event, bridge, event_queue):
    if not hasattr(autofocus_fn, 'ctr'):
        autofocus_fn.ctr = 0
    
    mmc = bridge.get_core()
    
    # autofocus
    try:
        mmc.full_focus()
    except:
        mmc.set_relative_position(5) # try moving up
        try:
            mmc.full_focus()
        except:
            mmc.set_relative_position(-10) # try moving down
            mmc.full_focus()
    
    time.sleep(1.0)
    pos = np.around(mmc.get_position(), decimals=3)
    
#     del_z = pos-event['z']
#     print(f'del_z: {del_z}')
    
    while autofocus_fn.ctr < num_events-1:
        for z_idx, z in enumerate(pos+z_range):
            new_event = deepcopy(events[autofocus_fn.ctr])
            new_event['axes']['z'] = z_idx
            new_event['z'] = z
            event_queue.put(new_event)
        
        if events[autofocus_fn.ctr]['axes'] == events[autofocus_fn.ctr+1]['axes']:
            # queue other channels at these axes
            autofocus_fn.ctr += 1
            continue
        else:
            # queue next axes
            event_queue.put(events[autofocus_fn.ctr+1])
            autofocus_fn.ctr += 1
            break
    else:
        # acquire z stack at last event
        for z_idx, z in enumerate(pos+z_range):
            new_event = deepcopy(events[autofocus_fn.ctr])
            new_event['axes']['z'] = z_idx
            new_event['z'] = z
            event_queue.put(new_event)

        event_queue.put(None) # stop acquisition


# def autofocus_fn(event, bridge, event_queue):
#     if not hasattr(autofocus_fn, 'ctr'):
#         autofocus_fn.ctr = 1
    
#     mmc = bridge.get_core()
    
#     mmc.full_focus(); time.sleep(0.2)
#     pos = np.around(mmc.get_position(), decimals=3)
    
#     del_z = pos-event['z']
#     print(f'del_z: {del_z}')
    
#     for z_idx, z in enumerate(pos+z_range):
#         new_event = deepcopy(event)
#         new_event['axes']['z'] = z_idx
#         new_event['z'] = z
#         event_queue.put(new_event)
    
#     # queue next event
#     if autofocus_fn.ctr < num_events:
#         event_queue.put(events[autofocus_fn.ctr])
#         autofocus_fn.ctr += 1
#     # stop acquisition
#     else:
#          event_queue.put(None)


def hook_fn(event, bridge, event_queue):
    if 'axes' in event.keys():
        if not hasattr(hook_fn, 'pos_index'):
            hook_fn.pos_index = event['axes']['position']
            autofocus_fn(event, bridge, event_queue)
            hook_fn.focused = True
            return
            
        if event['axes']['position'] != hook_fn.pos_index and not hook_fn.focused:
            autofocus_fn(event, bridge, event_queue)
            hook_fn.focused = True
            return
            
        hook_fn.pos_index = event['axes']['position']
        hook_fn.focused = False
    
#     print(event)
    return event

In [7]:
mmc.set_auto_shutter(True)
mmc.set_auto_focus_device('PFS')
mmc.set_config('Epi Channel', 'GFP EX466-40 EM525-50')

snap_live_manager = mmStudio.get_snap_live_manager()
if snap_live_manager.is_live_mode_on():
    snap_live_manager.set_live_mode_on(False)

acq = Acquisition(directory=r'D:\2022_02_10 Beads', name='pfs_test', post_hardware_hook_fn=hook_fn)
acq.acquire(events[0])